### 百度百科爬取

In [1]:
#url管理器
#coding:utf-8
class UrlManager(object):
    def __init__(self):
        self.new_urls=set()
        self.old_urls=set()
    def has_new_url(self):
        return self.new_url_size()!=0
    def get_new_url(self):
        new_url=self.new_urls.pop()
        self.old_urls.add(new_url)
        return new_url
    def add_new_url(self,url):
        if url is None:
            return
        if url not in self.new_urls and url not in self.old_urls:
            self.new_urls.add(url)
            
    def add_new_urls(self,urls):
        if urls is None or len(urls)==0:
            return
        for url in urls:
            self.add_new_url(url)
    def new_url_size(self):
        return len(self.new_urls)
    def old_url_size(self):
        return len(self.old_urls)
    #is ok

In [2]:
#html下载器
#coding:utf-8
import requests
class HtmlDownloader(object):
    def download(self,url):
        if url is None:
            return None
        user_agent='Mozilla/4.0(compatible;MSIE 5.5;Windows NT)'
        headers={'User-Agent':user_agent}
        r=requests.get(url,headers=headers)
        if r.status_code==200:
            r.encoding='utf-8'
            return r.text
        return None
            #is ok

In [4]:
#html解析器
#coding:utf-8
import re 
import urllib.parse as pa
from bs4 import BeautifulSoup
class HtmlParser(object):
    def parser2(self,page_url,html_cont):
        '''
        :param page_url: 下载页面的url
        :param html_cont: 下载页面的网页内容
        :return:返回url和数据
        '''
        if page_url is None or html_cont is None:
            return
        soup=BeautifulSoup(html_cont,'html.parser',from_encoding='UTF-8')
        print(soup)
        new_urls=self._get_new_urls(page_url,soup)
        new_data=self._get_new_data(page_url,soup)
        return new_urls,new_data
    
    def _get_new_urls(self,page_url,soup):
        '''
        抽取新的url集合
        :param page_url: 下载页面的url
        :param soup;soup
        :return:返回新的URL集合
        '''
        new_urls=set()
        #抽取符合要求的a标记
        links=soup.find_all('a',href=re.compile(r'\/item\/[0-9A-Za-z%]{1,100}\/[0-9]{0,100}'))
#         return links
#         print(links)
        for link in links:
            #提前href属性
            new_url=link['href']
#             print(links)
            #拼接成完整网站
#             print(page_url,'****',new_url)
            page_url='https://baike.baidu.com'
            new_full_url=pa.urljoin(page_url,new_url)
#             print(new_full_url)
            new_urls.add(new_full_url)
        return new_urls
    def _get_new_data (self,page_url,soup):
        '''
        抽取有效数据
        :param page_url: 下载页面的url
        :param soup:
        :return:返回有效数据
        '''
        data={}
        data['url']=page_url
        title=soup.find('dd',class_='lemmaWgt-lemmaTitle-title').find('h1')
        data['title']=title.get_text()
        summary=soup.find('div',class_='lemma-summary')
        data['summary']=summary.get_text()
        
        return data
    #?

In [5]:
#数据存储器
#coding:utf-8
import codecs
class DataOutput(object):
    def __init__(self):
        self.datas=[]
    def store_data(self,data):
        if data is None:
            return
        m={}
        for k in data:
            p=data[k].replace(u'\xa0', u' ')
            m.update({k:p})
        self.datas.append(m)
#         print(data)
        
    def output_html(self):
        fout=codecs.open('baike.html','w',encoding='gbk')
        fout.write('<html>')
        fout.write('<body>')
        fout.write('<table>')
        for data in self.datas:
            fout.write('<tr>')
            fout.write('<td>%s</td>'%data['url'])
            fout.write('<td>%s</td>'%data['title'])
            fout.write('<td>%s</td>'%data['summary'])
            fout.write('<tr>')
            self.datas.remove(data)
        fout.write('<table>')
        fout.write('<body>')
        fout.write('<html>')
        fout.close()     
        #is ok

In [6]:
class SpiderMan(object):
    def __init__(self):
        self.manager=UrlManager()
        self.downloader=HtmlDownloader()
        self.parser=HtmlParser()
        self.output=DataOutput()
    def crawl(self,root_url):
        #添加入口url
        self.manager.add_new_url(root_url)
        #判断url管理器 中是否有新的url，同时判断抓取了多少个url
        while (self.manager.has_new_url() and self.manager.old_url_size()<2):
#             try:
                #获取新的url
            new_url=self.manager.get_new_url()
                #下载网页
            html=self.downloader.download(new_url)
            print(new_url)
#             print(html)
                #抽取网页数据
            new_urls,data=self.parser.parser2(new_url,html)
#                 print(new_urls)
#                 print(data)
#             print(data)
                #添加到url管理器
            self.manager.add_new_urls(new_urls)
                #存储文件
            self.output.store_data(data)
            print('已经抓起%s个连接'%self.manager.old_url_size())
#             except Exception:
#                 print('crawl fail')
        self.output.output_html()

        
if __name__=='__main__':
    spider_man=SpiderMan()
    spider_man.crawl('https://baike.baidu.com/view/284853.htm')

https://baike.baidu.com/view/284853.htm
<!DOCTYPE html>

<!--STATUS OK-->
<html>
<head>
<meta charset="utf-8"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible">
<meta content="always" name="referrer"/>
<meta content="网络爬虫（又被称为网页蜘蛛，网络机器人，在FOAF社区中间，更经常的称为网页追逐者），是一种按照一定的规则，自动地抓取万维网信息的程序或者脚本。另外一些不常使用的名字还有蚂蚁、自动索引、模拟程序或者蠕虫。..." name="description"/>
<title>网络爬虫_百度百科</title>
<link href="/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="//www.baidu.com/img/baidu.svg" mask="" rel="icon" sizes="any"/>
<meta content="网络爬虫 爬虫 网页蜘蛛 爬虫程序 蜘蛛程序 搜索引擎蜘蛛 蜘蛛 网络蜘蛛 网络爬虫产生背景 网络爬虫面临的问题 网络爬虫分类 网络爬虫抓取目标分类 网络爬虫网页搜索策略 网络爬虫网页分析算法 网络爬虫补充" name="keywords"/>
<meta content="https://bkssl.bdimg.com/cms/static/baike.png" name="image"/>
<script type="text/javascript">
  // 配置 PD 监控。
  window.alogObjectConfig = {
    product: '103',
    page: '103_1',
    speed: {
      sample: '0.008'
    },
    monkey: {
      sample: '1',
      hid: '1533'
    },
    exception: { 
      sample: '0.004'
    },
    

C:\Users\dell\AppData\Local\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:146: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


https://baike.baidu.com/item/%E8%A0%95%E8%99%AB/4454380
<!DOCTYPE html>

<!--STATUS OK-->
<html>
<head>
<meta charset="utf-8"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible">
<meta content="always" name="referrer"/>
<meta content="蠕虫是一种能够利用系统漏洞通过网络进行自我传播的恶意程序。它是利用网络进行复制和传播，传染途径是通过网络和电子邮件。蠕虫病毒是自包含的程序(或是一套程序),它能传播它自身功能的拷贝或它的某些部分到其他的计算机系统中(通常是经过网络连接)。..." name="description"/>
<title>蠕虫（能通过网络进行自我传播的恶意程序）_百度百科</title>
<link href="/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="//www.baidu.com/img/baidu.svg" mask="" rel="icon" sizes="any"/>
<meta content="蠕虫 蠕虫定义 蠕虫入侵模式 蠕虫计算机蠕虫的通用结构 蠕虫发展历史 蠕虫与病毒的异同 蠕虫防治方法" name="keywords"/>
<meta content="https://bkssl.bdimg.com/cms/static/baike.png" name="image"/>
<script type="text/javascript">
  // 配置 PD 监控。
  window.alogObjectConfig = {
    product: '103',
    page: '103_1',
    speed: {
      sample: '0.008'
    },
    monkey: {
      sample: '1',
      hid: '1533'
    },
    exception: { 
      sample: '0.004'
    },
    feat

#### 结构化文本处理

In [23]:
#url管理器
#coding:utf-8
class UrlManager(object):
    def __init__(self):
        self.new_urls=set()
        self.old_urls=set()
    def has_new_url(self):
        return self.new_url_size()!=0
    def get_new_url(self):
        new_url=self.new_urls.pop()
        self.old_urls.add(new_url)
        return new_url
    def add_new_url(self,url):
        if url is None:
            return
        if url not in self.new_urls and url not in self.old_urls:
            self.new_urls.add(url)
            
    def add_new_urls(self,urls):
        if urls is None or len(urls)==0:
            return
        for url in urls:
            self.add_new_url(url)
    def new_url_size(self):
        return len(self.new_urls)
    def old_url_size(self):
        return len(self.old_urls)
    #is ok

In [24]:
#html下载器
#coding:utf-8
import requests
class HtmlDownloader(object):
    def download(self,url):
        if url is None:
            return None
        user_agent='Mozilla/4.0(compatible;MSIE 5.5;Windows NT)'
        headers={'User-Agent':user_agent}
        r=requests.get(url,headers=headers)
        if r.status_code==200:
            r.encoding='utf-8'
            return r.text
        return None
            #is ok

In [25]:
#html解析器
#coding:utf-8
import re 
import urllib.parse as pa
from bs4 import BeautifulSoup
class HtmlParser(object):
    def parser2(self,page_url,html_cont):
        '''
        :param page_url: 下载页面的url
        :param html_cont: 下载页面的网页内容
        :return:返回url和数据
        '''
        if page_url is None or html_cont is None:
            return
        soup=BeautifulSoup(html_cont,'html.parser',from_encoding='UTF-8')
#         print(soup)
        #抽取三元组的后面
#         the_end=soup.find_all(text=re.compile(r'(?<=value">\n)[\u4e00-\u9fa5 ]*?(?=\n<)'))
        the_end=re.findall(r'(?<=value">\n).*?(?=\n<)',html_cont)
        #抽取关系
#         relationship=soup.find_all(text=re.compile(r'(?<=name">)[\u4e00-\u9fa5 ]*?(?=<)'))
        relationship=re.findall(r'(?<=name">).*?(?=<)',html_cont)
        new_urls=self._get_new_urls(page_url,soup)
        new_data=self._get_new_data(page_url,soup)
        head=new_data['title']
        return new_urls,new_data,the_end,relationship,head
    
    def _get_new_urls(self,page_url,soup):
        '''
        抽取新的url集合
        :param page_url: 下载页面的url
        :param soup;soup
        :return:返回新的URL集合
        '''
        new_urls=set()
        #抽取符合要求的a标记
        links=soup.find_all('a',href=re.compile(r'\/item\/[0-9A-Za-z%]{1,100}\/[0-9]{0,100}'))
        
        #抽取三元组的后面
        the_end=soup.find_all(text=re.compile(r'(?<=value">\n)[\u4e00-\u9fa5 a-z]*?(?=\n<)'))
        #抽取关系
        relationship=soup.find_all(text=re.compile(r'(?<=name">)[\u4e00-\u9fa5 ]*?(?=<)'))
        for i in range(len(relationship)):
            print(relationship[i],"********************",the_end[i])
#         return links
#         print(links)
        for link in links:
            #提前href属性
            new_url=link['href']
#             print(links)
            #拼接成完整网站
#             print(page_url,'****',new_url)
            page_url='https://baike.baidu.com'
            new_full_url=pa.urljoin(page_url,new_url)
#             print(new_full_url)
            new_urls.add(new_full_url)
        return new_urls
    def _get_new_data (self,page_url,soup):
        '''
        抽取有效数据
        :param page_url: 下载页面的url
        :param soup:
        :return:返回有效数据
        '''
        data={}
        data['url']=page_url
        title=soup.find('dd',class_='lemmaWgt-lemmaTitle-title').find('h1')
        data['title']=title.get_text()
        summary=soup.find('div',class_='lemma-summary')
        data['summary']=summary.get_text()
        
        return data
    #?

In [26]:
#数据存储器
#coding:utf-8
import codecs
class DataOutput(object):
    def __init__(self):
        self.datas=[]
    def store_data(self,data):
        if data is None:
            return
        m={}
        for k in data:
            p=data[k].replace(u'\xa0', u' ')
            m.update({k:p})
        self.datas.append(m)
#         print(data)
        
    def output_html(self):
        fout=codecs.open('baike.html','w',encoding='gbk')
        fout.write('<html>')
        fout.write('<body>')
        fout.write('<table>')
        for data in self.datas:
            fout.write('<tr>')
            fout.write('<td>%s</td>'%data['url'])
            fout.write('<td>%s</td>'%data['title'])
            fout.write('<td>%s</td>'%data['summary'])
            fout.write('<tr>')
            self.datas.remove(data)
        fout.write('<table>')
        fout.write('<body>')
        fout.write('<html>')
        fout.close()     
        #is ok

In [32]:
import pandas as pd
class SpiderMan(object):
    def __init__(self):
        self.manager=UrlManager()
        self.downloader=HtmlDownloader()
        self.parser=HtmlParser()
        self.output=DataOutput()
    def crawl(self,root_url):
        #添加入口url
        self.manager.add_new_url(root_url)
        #判断url管理器 中是否有新的url，同时判断抓取了多少个url
        datas=[]
        header=[]
        while (self.manager.has_new_url() and self.manager.old_url_size()<3000):
            try:
                #获取新的url
                new_url=self.manager.get_new_url()
                #下载网页
                html=self.downloader.download(new_url)
                print(new_url)
#             print(html)
                #抽取网页数据
                new_urls,data,the_end,relation,head=self.parser.parser2(new_url,html)
                if head not in header:
                    header.append(head)
                    if len(the_end)<=len(relation):
                        for i in range(len(the_end)):
                            a=re.compile(r'<.*?>')
                            the_end[i]=a.sub('',the_end[i])
                            relation[i]=relation[i].replace('&nbsp;','')
                            k=[head,relation[i],the_end[i]]
                            print(k)
#                     data.append(head)
#                     data.append(relation)
#                     data.append(the_end)
                            datas.append(k)
                    else:
                        for i in range(len(relation)):
                            a=re.compile(r'<.*?>')
                            the_end[i]=a.sub('',the_end[i])
                            relation[i]=relation[i].replace('&nbsp;','')
                            k=[head,relation[i],the_end[i]]
                            print(k)
#                     data.append(head)
#                     data.append(relation)
#                     data.append(the_end)
                            datas.append(k)
#                     print(head,"&&&&&&&&&&&&&&&&&&&&",relation[i],'&&&&&&&&&&&&&&&&&&&&',the_end[i])
                else:
                    pass
#                 print(new_urls)
#                 print(data)
#             print(data)
                #添加到url管理器
                self.manager.add_new_urls(new_urls)
                #存储文件
                self.output.store_data(data)
                print('已经抓起%s个连接'%self.manager.old_url_size())
            
            except Exception:
                print('crawl fail')
        output=pd.DataFrame(datas)
#         print(output)
        output.to_csv('知识图谱.csv')
#         self.output.output_html()

        
if __name__=='__main__':
    spider_man=SpiderMan()
    spider_man.crawl('https://baike.baidu.com/view/284853.htm')

https://baike.baidu.com/view/284853.htm
['网络爬虫', '中文名', '网络爬虫']
['网络爬虫', '外文名', 'web crawler']
['网络爬虫', '别称', '网络蜘蛛']
['网络爬虫', '目的', '按要求获取万维网信息']
已经抓起1个连接


C:\Users\dell\AppData\Local\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:146: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


https://baike.baidu.com/item/%E4%B8%87%E7%BB%B4%E7%BD%91/215515
['www', '中文名', '万维网']
['www', '外文名', 'World Wide Web']
['www', '其他称呼', '环球信息网']
['www', '外语简称', 'WWW\u200b或Web']
['www', '包括', 'Web客户端和Web服务器']
已经抓起2个连接
https://baike.baidu.com/item/%E8%B6%85%E6%96%87%E6%9C%AC/2832422
['超文本', '中文名', '超级文本']
['超文本', '外文名', 'HyperText']
['超文本', '中文缩写', '超文本']
['超文本', '外语缩写', 'HT']
已经抓起3个连接
https://baike.baidu.com/item/%E6%A0%87%E5%87%86%E9%80%9A%E7%94%A8%E6%A0%87%E8%AE%B0%E8%AF%AD%E8%A8%80/6805073
['标准通用置标语言', '中文名', '标准通用标记语言']
['标准通用置标语言', '外文名', 'Standard Generalized Markup language']
['标准通用置标语言', '中文', '通用标言']
['标准通用置标语言', '', 'SGM或SGML（都作文件扩展名）']
['标准通用置标语言', '类型', '标记语言']
['标准通用置标语言', '其他称呼', '~标注语言、~置标语言~标识语言']
['标准通用置标语言', '基本思想', '分开文档的内容与样式']
['标准通用置标语言', '可选参数', 'SGML-bctf、SGML-boot、charset']
已经抓起4个连接
https://baike.baidu.com/item/www/20192985#viewPageContent
已经抓起5个连接
https://baike.baidu.com/item/www/109924#viewPageContent
已经抓起6个连接
https://baike.baidu.com/item/%E7%94%B5%E5%AD%90%E6

https://baike.baidu.com/item/%E8%B5%84%E6%9C%AC%E4%B8%BB%E4%B9%89/87961
['资本主义', '中文名', '资本主义']
['资本主义', '外文名', 'capitalism']
['资本主义', '萌芽', '文艺复兴时期']
['资本主义', '特点', '私有制']
['资本主义', '统治阶级', '资产阶级']
已经抓起31个连接
https://baike.baidu.com/item/%E7%BB%84%E7%BB%87%E8%80%85/3188042
['组织者', '中文名', '先行组织者']
['组织者', '外文名', 'an organizer']
['组织者', '类别', '名词']
['组织者', '释义', '某一个方面内容起组织作用']
已经抓起32个连接
https://baike.baidu.com/item/%E6%B3%B0%E7%B1%B3%E5%B0%94%E6%97%8F/6720814
['泰米尔族', '中文名', '泰米尔族']
['泰米尔族', '主要分布', '印度的南部和斯里兰卡的东北部']
['泰米尔族', '相关组织', '泰米尔猛虎组织']
['泰米尔族', '擅长', '游击战术']
已经抓起33个连接
https://baike.baidu.com/item/%E7%99%BD%E4%BF%84%E7%BD%97%E6%96%AF/84249
['白俄罗斯共和国', '中文名称', 'Republic of Belarus']
['白俄罗斯共和国', '英文名称', '白俄罗斯']
['白俄罗斯共和国', '简称', '欧洲']
['白俄罗斯共和国', '所属洲', '布列斯特、维捷布斯克、戈梅利等']
['白俄罗斯共和国', '首都', '1944年7月3日']
['白俄罗斯共和国', '主要城市', '《我们白俄罗斯人》']
['白俄罗斯共和国', '国庆日', 'BLR']
['白俄罗斯共和国', '国歌', 'UTC+2']
['白俄罗斯共和国', '国家代码', '总统制共和制']
['白俄罗斯共和国', '官方语言', '总统：亚历山大·格里戈里耶维奇·卢卡申科']
['白俄罗斯共和国', '货币', '白俄罗

['荷兰语', '中文名', '荷兰语']
['荷兰语', '外文名', 'Dutch']
['荷兰语', '语系', '“印欧语系日耳曼语族西支”']
['荷兰语', '母语使用', '两千二百万']
['荷兰语', '使用地区', '荷兰、比利时、苏里南']
['荷兰语', '作为官方语言', '荷兰、比利时、苏里南']
已经抓起58个连接
https://baike.baidu.com/item/%E5%AE%BE%E5%A4%95%E6%B3%95%E5%B0%BC%E4%BA%9A%E8%BD%A6%E7%AB%99/15516830
['宾夕法尼亚车站', '中文名', '纽约宾夕法尼亚车站']
['宾夕法尼亚车站', '外文名', 'New York Pennsylvania Station']
['宾夕法尼亚车站', '简称', '纽约宾州车站']
['宾夕法尼亚车站', '英文简称', 'Penn Station']
['宾夕法尼亚车站', '位置', '美国纽约州纽约曼哈顿中城']
['宾夕法尼亚车站', '客流量', '每日300,000人次以上']
['宾夕法尼亚车站', '所有权', '美铁']
['宾夕法尼亚车站', '营运权', '美铁']
['宾夕法尼亚车站', '其他使用者', '新泽西捷运，纽约大都会运输署']
['宾夕法尼亚车站', '车站代码', 'NYP（铁路），ZYP（IATA）']
['宾夕法尼亚车站', '月台数', '11']
['宾夕法尼亚车站', '股道数', '21']
['宾夕法尼亚车站', '经过路线数', '16条（美铁）']
['宾夕法尼亚车站', '启用年份', '1910年']
['宾夕法尼亚车站', '地理坐标', "40º45'02''N,73º59'38''W"]
已经抓起59个连接
https://baike.baidu.com/item/%E9%B8%A1%E8%9B%8B/6405
['鸡蛋', '中文名', '鸡蛋']
['鸡蛋', '外文名', '(hen&#39;s) egg']
['鸡蛋', '别名', '鸡子，鸡卵，滚头、剥之、甩果等']
['鸡蛋', '性味', '味甘，性平，无毒']
已经抓起60个连接
https://baike.baidu.com/item/%E5%8D

...

https://baike.baidu.com/item/%E8%B7%AF%E6%98%93%E6%96%AF%C2%B7%E8%92%99%E5%B7%B4%E9%A1%BF/5425188
['路易斯·蒙巴顿', '中文名', '路易斯·弗朗西斯·艾伯特·维克托·尼古拉斯·蒙巴顿']
['路易斯·蒙巴顿', '外文名', 'Louis Francis Albert Victor Nicholas Mountbatten']
['路易斯·蒙巴顿', '别名', '蒙巴顿伯爵']
['路易斯·蒙巴顿', '国籍', '英国']
['路易斯·蒙巴顿', '出生地', '英国温莎']
['路易斯·蒙巴顿', '出生日期', '1900年（庚子年）6月25日']
['路易斯·蒙巴顿', '逝世日期', '1979年（己未年）8月27日']
['路易斯·蒙巴顿', '职业', '英国海军元帅，东南亚盟军总司令，印度总督']
['路易斯·蒙巴顿', '毕业院校', '奥斯本皇家海军学校、朴次茅斯通信学校、达特茅斯皇家海军学院']
['路易斯·蒙巴顿', '信仰', '圣公宗']
['路易斯·蒙巴顿', '主要成就', '维多利亚皇家爵级大十字勋章']
已经抓起2863个连接
https://baike.baidu.com/item/%E4%BA%BA%E6%B0%91%E7%9A%84%E8%B4%A2%E4%BA%A7/22426813
['人民的财产', '中文名', '人民的财产']
['人民的财产', '其它译名', '天凉好个秋']
['人民的财产', '出品公司', '上海耀客传媒股份有限公司']
['人民的财产', '拍摄地点', '沈严、刘海波']
['人民的财产', '导演', '周梅森']
['人民的财产', '编剧', '靳东，闫妮，陈晓，黄志忠，秦岚，赵立新']
['人民的财产', '主演', '60集']
已经抓起2864个连接
https://baike.baidu.com/item/%E7%AC%AC%E4%B8%89%E4%BA%BA%E7%A7%B0%E5%8D%95%E6%95%B0/1586952
['第三人称单数', '中文名', '第三人称单数']
['第三人称单数', '外文名', 'Third-person singular']

https://baike.baidu.com/item/%E6%B0%B8%E5%98%89/11009228
['永嘉', '中文名', '永嘉']
['永嘉', '性质', '帝王年号']
['永嘉', '朝代', '东汉、西晋、大理国']
['永嘉', '代表', '永嘉之乱']
已经抓起2889个连接
https://baike.baidu.com/item/%E7%A0%81%E6%B5%81/8660022
['码流', '中文名', '码流']
['码流', '外文名', 'Data Rate']
['码流', '别名', '码率']
['码流', '用途', '视频编码中画面质量控制中部分']
['码流', '领域', '媒体通讯']
['码流', '计算公式', '码流=采样率×比特数×声道']
['码流', '单位', 'bps']
已经抓起2890个连接
https://baike.baidu.com/item/%E9%A9%BB%E9%A9%AC%E5%BA%97/140955
['驻马店', '中文名称', '驻马店']
['驻马店', '外文名称', 'Zhumadian City']
['驻马店', '别名', '汝宁、蔡州、天中']
['驻马店', '行政区类别', '地级市']
['驻马店', '所属地区', '中国华中']
['驻马店', '下辖地区', '1个区、9个县']
['驻马店', '政府驻地', '驿城区']
['驻马店', '电话区号', '0396']
['驻马店', '邮政区码', '463000']
['驻马店', '地理位置', '黄淮平原']
['驻马店', '面积', '15083平方公里']
['驻马店', '人口', '中原官话-蔡鲁片-漯项小片（泌阳县为中原官话南鲁片）']
['驻马店', '方言', '温带季风气候']
['驻马店', '气候条件', '南海禅寺、嵖岈山、杨靖宇纪念馆、薄山湖']
['驻马店', '著名景点', '确山李新店机场、平舆通用机场']
['驻马店', '机场', '驻马店火车站、驻马店西站']
['驻马店', '火车站', '豫Q']
['驻马店', '车牌代码', '东经113°10 - - 115°12']
['驻马店', '经度', "北纬32°18' -- 3

https://baike.baidu.com/item/%E5%A4%A7%E5%9C%B0%E9%81%97%E5%AD%A4/3140957
['大地遗孤', '中文名', '大地遗孤']
['大地遗孤', '导演', '贺志良']
['大地遗孤', '集数', '30集']
['大地遗孤', '颜色', '彩色']
已经抓起2916个连接
https://baike.baidu.com/item/%E7%9F%B3%E7%A0%81%E8%A1%97%E9%81%93/5613780
['石码街道', '中文名称', '石码街道']
['石码街道', '地理位置', '福建东南部九龙江下游']
['石码街道', '面积', '5.26平方公里']
['石码街道', '人口', '66735人']
已经抓起2917个连接
https://baike.baidu.com/item/%E8%B5%B5%E5%BF%97%E5%87%8C/4836044
['赵志凌', '中文名', '赵志凌']
['赵志凌', '国籍', '中国']
['赵志凌', '民族', '汉']
['赵志凌', '出生日期', '1943']
['赵志凌', '职业', '功夫片演员']
['赵志凌', '代表作品', '《蛇形刁手》和《功夫》']
['赵志凌', '身份', '林世荣徒孙']
已经抓起2918个连接
https://baike.baidu.com/item/%E6%97%A5%E6%9C%88%E6%B9%96/15884063#viewPageContent
已经抓起2919个连接
https://baike.baidu.com/item/%E5%B0%A7%E5%B1%B1/30534
['尧山', '中文名称', '尧山风景名胜区']
['尧山', '外文名称', 'YaoShan scenic spot']
['尧山', '别名', '中原独秀——石人山风景名胜区']
['尧山', '所属地区', '中国华中']
['尧山', '电话区号', '0375']
['尧山', '邮政区码', '467343']
['尧山', '地理位置', '位于河南省平顶山市鲁山县西']
['尧山', '面积', '景区总面积268平方公里']
['尧山', '气候条件', '温

https://baike.baidu.com/item/%E4%BC%9A%E5%B1%95/4465881
['会展', '中文名', '会展']
['会展', '外文名', 'Exhibition']
['会展', '内容', '博览会、展销活动、大中小型会议']
['会展', '起源', '暂无考证']
已经抓起2946个连接
https://baike.baidu.com/item/%E9%AB%98%E4%B8%BD/19696402#viewPageContent
已经抓起2947个连接
https://baike.baidu.com/item/%E9%BB%91%E4%B8%89%E8%A7%92/7419
['黑三角', '中文名', '黑三角']
['黑三角', '外文名', 'Black triangle']
['黑三角', '其它译名', '《义无反顾》《一号罪恶》《生死与共》《背叛》']
['黑三角', '制片地区', '中国大陆']
['黑三角', '导演', '史晨风']
['黑三角', '主演', '孙红雷，柳云龙，胡军，许晴']
['黑三角', '上映时间', '2001']
['黑三角', '语言', '普通话']
已经抓起2948个连接
https://baike.baidu.com/item/%E8%A5%BF%E9%87%8C%E8%A5%BF%E4%BA%9A%E6%88%98%E4%BA%89/2683283
['西里西亚战争', '名称', '西里西亚战争']
['西里西亚战争', '地点', '西里西亚']
['西里西亚战争', '时间', '1740年12月']
['西里西亚战争', '参战方', '普鲁士']
已经抓起2949个连接
https://baike.baidu.com/item/%E5%B8%83%E5%B0%94%E4%BB%80%E7%BB%B4%E5%85%8B%E5%85%9A/1900165
['布尔什维克党', '中文名', '布尔什维克党']
['布尔什维克党', '外文名', 'Bolshevik']
['布尔什维克党', '创立者', '列宁']
['布尔什维克党', '类型', '无产阶级政党']
['布尔什维克党', '俄文名', 'Большевики́']
已经抓起2950个

https://baike.baidu.com/item/%E5%85%B3%E5%B1%B1/3689727
['关山', '中文名称', '关山']
['关山', '外文名称', 'Kuanshan']
['关山', '地理位置', '新乡']
['关山', '开放时间', '全天']
已经抓起2976个连接
https://baike.baidu.com/item/%E8%90%A8%E7%A6%8F%E5%85%8B%E9%83%A1/7920363
['萨福克郡', '中文名', '萨福克郡']
['萨福克郡', '外文名', 'Suffolk']
['萨福克郡', '人口', '702,100']
['萨福克郡', '面积', '3,801平方公里']
已经抓起2977个连接
https://baike.baidu.com/item/%E8%BF%AA%E6%B2%B3/4816807
['迪河', '中文名', '迪河']
['迪河', '源出海拔', '1,250公尺']
['迪河', '全长', '约145公里']
['迪河', '所在国家', '苏格兰']
已经抓起2978个连接
https://baike.baidu.com/item/%E5%9F%83%E5%B0%94%E6%96%87%C2%B7%E7%BA%A6%E7%BF%B0%E9%80%8A/13881796#viewPageContent
已经抓起2979个连接
https://baike.baidu.com/item/%E7%9A%87%E5%90%8E/1080437
['皇后', '中文名', '皇后']
['皇后', '外文名', 'empress']
['皇后', '拼音', 'huáng hòu']
['皇后', '别名', '皇帝正妻']
['皇后', '自称', '对上称梓童，对下称本宫']
['皇后', '著名皇后', '女']
已经抓起2980个连接
https://baike.baidu.com/item/%E8%B0%8F%E4%BA%89/4720635
['谏争', '中文名', '谏争']
['谏争', '拼音', 'jiàn zhēng&nbsp;']
['谏争', '释义', '直言规劝。']
已经抓起2981个连接
https://baike.

In [10]:
import pandas as pd
pa=pd.read_csv('KnowledgeGraph.csv',index_col=0)
pa


,0,1,2
0,网络爬虫,中文名,网络爬虫
1,网络爬虫,外文名,web crawler
2,网络爬虫,别称,网络蜘蛛
3,网络爬虫,目的,按要求获取万维网信息
4,www,中文名,万维网
5,www,外文名,World Wide Web
6,www,其他称呼,环球信息网
7,www,外语简称,WWW​或Web
8,www,包括,Web客户端和Web服务器
9,超文本,中文名,超级文本


In [17]:
pa.iloc[:,0].value_counts().count()

2382

### 非结构化文本处理 

In [18]:
#url管理器
#coding:utf-8
class UrlManager(object):
    def __init__(self):
        self.new_urls=set()
        self.old_urls=set()
    def has_new_url(self):
        return self.new_url_size()!=0
    def get_new_url(self):
        new_url=self.new_urls.pop()
        self.old_urls.add(new_url)
        return new_url
    def add_new_url(self,url):
        if url is None:
            return
        if url not in self.new_urls and url not in self.old_urls:
            self.new_urls.add(url)
            
    def add_new_urls(self,urls):
        if urls is None or len(urls)==0:
            return
        for url in urls:
            self.add_new_url(url)
    def new_url_size(self):
        return len(self.new_urls)
    def old_url_size(self):
        return len(self.old_urls)
    #is ok

In [19]:
#html下载器
#coding:utf-8
import requests
class HtmlDownloader(object):
    def download(self,url):
        if url is None:
            return None
        user_agent='Mozilla/4.0(compatible;MSIE 5.5;Windows NT)'
        headers={'User-Agent':user_agent}
        r=requests.get(url,headers=headers)
        if r.status_code==200:
            r.encoding='utf-8'
            return r.text
        return None
            #is ok

In [26]:
#html解析器
#coding:utf-8
import re 
import urllib.parse as pa
from bs4 import BeautifulSoup
class HtmlParser(object):
    def parser2(self,page_url,html_cont):
        '''
        :param page_url: 下载页面的url
        :param html_cont: 下载页面的网页内容
        :return:返回url和数据
        '''
        if page_url is None or html_cont is None:
            return
        soup=BeautifulSoup(html_cont,'html.parser',from_encoding='UTF-8')
#         print(soup)
        new_urls=self._get_new_urls(page_url,soup)
        new_data=self._get_new_data(page_url,soup)
        return new_urls,new_data
    
    def _get_new_urls(self,page_url,soup):
        '''
        抽取新的url集合
        :param page_url: 下载页面的url
        :param soup;soup
        :return:返回新的URL集合
        '''
        new_urls=set()
        #抽取符合要求的a标记
        links=soup.find_all('a',href=re.compile(r'\/item\/[0-9A-Za-z%]{1,100}\/[0-9]{0,100}'))
#         return links
#         print(links)
        for link in links:
            #提前href属性
            new_url=link['href']
#             print(links)
            #拼接成完整网站
#             print(page_url,'****',new_url)
            page_url='https://baike.baidu.com'
            new_full_url=pa.urljoin(page_url,new_url)
#             print(new_full_url)
            new_urls.add(new_full_url)
        return new_urls
    def _get_new_data (self,page_url,soup):
        '''
        抽取有效数据
        :param page_url: 下载页面的url
        :param soup:
        :return:返回有效数据
        '''
        data={}
        data['url']=page_url
        title=soup.find('dd',class_='lemmaWgt-lemmaTitle-title').find('h1')
        data['title']=title.get_text()
        summary=soup.find('div',class_='lemma-summary')
        data['summary']=summary.get_text()
        
        return data
    #?

In [27]:
#数据存储器
#coding:utf-8
import codecs
class DataOutput(object):
    def __init__(self):
        self.datas=[]
    def store_data(self,data):
        if data is None:
            return
        m={}
        for k in data:
            p=data[k].replace(u'\xa0', u' ')
            m.update({k:p})
        self.datas.append(m)
#         print(data)
        
    def output_html(self):
        fout=codecs.open('baike.html','w',encoding='gbk')
        fout.write('<html>')
        fout.write('<body>')
        fout.write('<table>')
        for data in self.datas:
            fout.write('<tr>')
            fout.write('<td>%s</td>'%data['url'])
            fout.write('<td>%s</td>'%data['title'])
            fout.write('<td>%s</td>'%data['summary'])
            fout.write('<tr>')
            self.datas.remove(data)
        fout.write('<table>')
        fout.write('<body>')
        fout.write('<html>')
        fout.close()     
        #is ok

In [29]:
import pandas as pd
class SpiderMan(object):
    def __init__(self):
        self.manager=UrlManager()
        self.downloader=HtmlDownloader()
        self.parser=HtmlParser()
        self.output=DataOutput()
    def crawl(self,root_url):
        #添加入口url
        self.manager.add_new_url(root_url)
        #判断url管理器 中是否有新的url，同时判断抓取了多少个url
        p=[]
        while (self.manager.has_new_url() and self.manager.old_url_size()<3000):
            try:
                #获取新的url
                new_url=self.manager.get_new_url()
                #下载网页
                html=self.downloader.download(new_url)
                print(new_url)
#             print(html)
                #抽取网页数据
                new_urls,data=self.parser.parser2(new_url,html)
#                 print(new_urls)
#                 print(data)
#             print(data)
                #添加到url管理器
                self.manager.add_new_urls(new_urls)
                #存储文件
                u=[data['title'],data['summary']]
                p.append(u)
#                 self.output.store_data(data)
                print('已经抓起%s个连接'%self.manager.old_url_size())
            except Exception:
                print('crawl fail')
        kkk=pd.DataFrame(p)
        kkk.to_csv('非结构化摘要.csv')
#         self.output.output_html()

        
if __name__=='__main__':
    spider_man=SpiderMan()
    spider_man.crawl('https://baike.baidu.com/view/284853.htm')

https://baike.baidu.com/view/284853.htm
已经抓起1个连接


C:\Users\dell\AppData\Local\Continuum\anaconda3\lib\site-packages\bs4\__init__.py:146: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


https://baike.baidu.com/item/%E8%A0%95%E8%99%AB/4454380
已经抓起2个连接
https://baike.baidu.com/item/%E8%A0%95%E8%99%AB/21038#viewPageContent
已经抓起3个连接
https://baike.baidu.com/item/%E4%B8%87%E7%BB%B4%E7%BD%91/215515
已经抓起4个连接
https://baike.baidu.com/item/%E7%BB%9F%E4%B8%80%E8%B5%84%E6%BA%90%E6%A0%87%E8%AF%86%E7%AC%A6/2890807
已经抓起5个连接
https://baike.baidu.com/item/%E8%8B%B1%E8%AF%AD/109997
已经抓起6个连接
https://baike.baidu.com/item/%E7%8E%AF%E7%90%83%E4%BF%A1%E6%81%AF%E7%BD%91/9377238
已经抓起7个连接
https://baike.baidu.com/item/%E4%B8%8D%E5%88%97%E9%A2%A0%E7%BE%A4%E5%B2%9B/6312089
已经抓起8个连接
https://baike.baidu.com/item/%E8%8B%B1%E8%AF%AD/19915502
已经抓起9个连接
https://baike.baidu.com/item/%E5%A8%81%E5%B0%94%E5%A3%AB%E8%AF%AD/4743146
已经抓起10个连接
https://baike.baidu.com/item/%E4%BD%8E%E5%9C%B0%E6%92%92%E5%85%8B%E9%80%8A%E8%AF%AD/9104730
已经抓起11个连接
https://baike.baidu.com/item/%E5%9B%BD%E5%AE%B6/17205
已经抓起12个连接
https://baike.baidu.com/item/%E5%B7%A5%E4%BD%9C%E8%AF%AD%E8%A8%80/15714728
已经抓起13个连接
https://baike.baidu.com/

https://baike.baidu.com/item/%E5%BC%A0%E8%BE%85/20716
已经抓起107个连接
https://baike.baidu.com/item/%E6%94%BF%E5%BA%9C/1416952
已经抓起108个连接
https://baike.baidu.com/item/%E9%AB%98%E7%AD%89%E6%95%B0%E5%AD%A6/10790565#viewPageContent
已经抓起109个连接
https://baike.baidu.com/item/%E5%8D%97%E6%B5%A6/5579739
已经抓起110个连接
https://baike.baidu.com/item/%E4%B8%AD%E5%BF%83%E5%9F%8E%E5%B8%82/10849239
已经抓起111个连接
https://baike.baidu.com/item/%E5%85%B0%E5%B7%9E/170826
已经抓起112个连接
https://baike.baidu.com/item/%E5%AD%99%E4%B8%AD%E5%B1%B1/128084
已经抓起113个连接
https://baike.baidu.com/item/%E8%87%AA%E7%94%B1/1296760#viewPageContent
已经抓起114个连接
https://baike.baidu.com/item/%E4%BD%95%E4%BC%91/643109
已经抓起115个连接
https://baike.baidu.com/item/%E5%85%B0%E5%B7%9E%E5%A4%96%E8%AF%AD%E8%81%8C%E4%B8%9A%E5%AD%A6%E9%99%A2/2968290
已经抓起116个连接
https://baike.baidu.com/item/%E4%B8%A4%E9%99%A2%E9%99%A2%E5%A3%AB/2247082
已经抓起117个连接
https://baike.baidu.com/item/%E8%B4%BA%E5%BE%AA/4246571
已经抓起118个连接
https://baike.baidu.com/item/%E8%A5%BF%E5%8C%97%E5

https://baike.baidu.com/item/%E6%9C%88%E7%89%99%E6%B9%96/10474336
已经抓起209个连接
https://baike.baidu.com/item/%E5%BE%B7%E6%84%8F%E5%BF%97%E5%B8%9D%E5%9B%BD/1902
已经抓起210个连接
https://baike.baidu.com/item/%E8%81%8C%E4%B8%9A%E6%8A%80%E6%9C%AF%E5%AD%A6%E6%A0%A1/1593871
已经抓起211个连接
https://baike.baidu.com/item/%E5%AD%99%E6%85%8E/14191700
已经抓起212个连接
https://baike.baidu.com/item/%E7%89%99%E4%B9%B0%E5%8A%A0%E6%9D%A5%E4%BF%A1/3329510
已经抓起213个连接
https://baike.baidu.com/item/%E4%BB%81%E5%8C%96%E5%8E%BF/3335574
已经抓起214个连接
https://baike.baidu.com/item/%E6%9F%A5%E7%90%86%E6%9B%BC%E5%A4%A7%E5%B8%9D/18859546
已经抓起215个连接
https://baike.baidu.com/item/%E5%90%B4%E8%83%9C%E5%88%A9/11001938
已经抓起216个连接
https://baike.baidu.com/item/%E5%A4%A9%E6%96%87%E5%AD%A6/402068
已经抓起217个连接
https://baike.baidu.com/item/%E8%87%AA%E7%84%B6%E4%B8%BB%E4%B9%89/806927
已经抓起218个连接
https://baike.baidu.com/item/%E8%B4%9D%E5%A5%A5%E6%AD%A6%E5%A4%AB/24730
已经抓起219个连接
https://baike.baidu.com/item/%CE%B2/9134749
已经抓起220个连接
https://baike.baidu.co

...

https://baike.baidu.com/item/%E5%B1%B1%E6%B5%B7%E7%BB%8F/20400146#viewPageContent
已经抓起2560个连接
https://baike.baidu.com/item/%E5%B0%A4%E5%8B%87/8697
已经抓起2561个连接
https://baike.baidu.com/item/%E5%88%98%E6%98%BE/13779640
已经抓起2562个连接
https://baike.baidu.com/item/%E9%AD%94%E6%9C%AF%E5%B8%88/31047
已经抓起2563个连接
https://baike.baidu.com/item/%E9%BA%A6%E5%8D%8E%E4%B8%89/532449
已经抓起2564个连接
https://baike.baidu.com/item/%E9%AB%98%E7%BF%8A%E6%B5%9A/9883752
已经抓起2565个连接
https://baike.baidu.com/item/%E8%87%A3%E5%83%9A/9488148
已经抓起2566个连接
https://baike.baidu.com/item/%E5%A4%A9%E4%B8%8B%E5%9B%9B%E8%81%9A/8028492
已经抓起2567个连接
https://baike.baidu.com/item/%E5%8D%9A%E5%A3%AB%E5%90%8E%E7%A7%91%E7%A0%94%E6%B5%81%E5%8A%A8%E7%AB%99/7310663
已经抓起2568个连接
https://baike.baidu.com/item/atp/10948507#viewPageContent
已经抓起2569个连接
https://baike.baidu.com/item/%E4%BD%90%E5%8F%A4%E6%AD%A3%E4%BA%BA/16946837
已经抓起2570个连接
https://baike.baidu.com/item/%E7%83%AD%E5%B8%A6%E5%AD%A3%E9%A3%8E%E6%B0%94%E5%80%99/1130243
已经抓起2571个连接
https:/

https://baike.baidu.com/item/%E6%B5%B7%E5%B2%AD/1698319
已经抓起2659个连接
https://baike.baidu.com/item/%E6%9D%8E%E5%BB%BA%E4%B8%AD/8594682#viewPageContent
已经抓起2660个连接
https://baike.baidu.com/item/%E8%B4%BE%E5%AE%AA/363000
已经抓起2661个连接
https://baike.baidu.com/item/%E5%8D%B0%E7%AC%AC%E5%AE%89%E4%BA%BA/254718
已经抓起2662个连接
https://baike.baidu.com/item/%E5%9B%BD%E5%8A%A1%E5%A7%94%E5%91%98/6095454
已经抓起2663个连接
https://baike.baidu.com/item/%E9%B3%91%E9%B2%8F/971179
已经抓起2664个连接
https://baike.baidu.com/item/%E5%85%84%E5%BC%9F/66#viewPageContent
已经抓起2665个连接
https://baike.baidu.com/item/%E5%BC%A0%E4%BA%9A%E5%BF%A0/4795753
已经抓起2666个连接
https://baike.baidu.com/item/%E5%BF%83%E7%9A%84%E6%96%B9%E5%90%91/1567861
已经抓起2667个连接
https://baike.baidu.com/item/%E5%90%89%E5%86%88%E6%98%8C%E4%BB%81/19328170
已经抓起2668个连接
https://baike.baidu.com/item/%E6%97%A0%E9%99%90%E7%94%9F%E6%9C%BA/1510353
已经抓起2669个连接
https://baike.baidu.com/item/%E7%99%BD%E5%86%B0/10967
已经抓起2670个连接
https://baike.baidu.com/item/%E8%B5%B5%E6%99%AE/155
已

https://baike.baidu.com/item/%E9%BB%91%E4%BA%BA/2634720#viewPageContent
已经抓起2760个连接
https://baike.baidu.com/item/%E8%A5%BF%E5%93%88%E5%8A%AA%E5%85%8B/63202
已经抓起2761个连接
https://baike.baidu.com/item/%E4%B8%AD%E9%A4%90%E5%8E%85/20834545
已经抓起2762个连接
https://baike.baidu.com/item/%E7%BD%97%E9%A9%AC%E5%86%85%E6%88%98/2683323
已经抓起2763个连接
https://baike.baidu.com/item/%E7%A0%82%E4%BB%81/581130
已经抓起2764个连接
https://baike.baidu.com/item/%E5%8B%87%E5%A3%AB/19607463
已经抓起2765个连接
https://baike.baidu.com/item/%E5%88%98%E5%BF%97%E4%B8%B9/892
已经抓起2766个连接
https://baike.baidu.com/item/%E6%BD%98%E7%A4%BC%E5%B9%B3/16449039
已经抓起2767个连接
https://baike.baidu.com/item/%E5%8F%9B%E5%BE%92/7355965
已经抓起2768个连接
https://baike.baidu.com/item/%E4%B8%9C%E7%9B%9F%E7%A7%98%E4%B9%A6%E5%A4%84/2837764
已经抓起2769个连接
https://baike.baidu.com/item/%E9%A1%B9%E7%BE%BD/23180073#viewPageContent
已经抓起2770个连接
https://baike.baidu.com/item/%E9%A3%9E%E5%BB%89/5770314#viewPageContent
已经抓起2771个连接
https://baike.baidu.com/item/%E6%B2%99%E5%A4%B4%E

已经抓起2861个连接
https://baike.baidu.com/item/%E4%B8%81%E9%A6%99/6069
已经抓起2862个连接
https://baike.baidu.com/item/%E7%BA%A2%E5%AD%A9%E5%84%BF/6860314
已经抓起2863个连接
https://baike.baidu.com/item/%E7%9C%9F%E8%B0%9B/74122
已经抓起2864个连接
https://baike.baidu.com/item/%E6%96%B0%E5%9F%8E%E8%A1%97%E9%81%93/10410910#viewPageContent
已经抓起2865个连接
https://baike.baidu.com/item/%E6%9D%A8%E8%BD%B6/17192650
已经抓起2866个连接
https://baike.baidu.com/item/%E8%96%9B%E8%9F%A0/6518096
已经抓起2867个连接
https://baike.baidu.com/item/%E6%99%AE%E9%B2%81%E5%A3%AB%E7%8E%8B%E5%9B%BD/4135055
已经抓起2868个连接
https://baike.baidu.com/item/%E8%8A%B3%E5%AE%98/6517184
已经抓起2869个连接
https://baike.baidu.com/item/%E6%A3%AE%E5%B7%9D%E6%99%BA%E4%B9%8B/217361
已经抓起2870个连接
https://baike.baidu.com/item/%E6%96%B0%E5%B0%91%E6%9E%97%E4%BA%94%E7%A5%96/4719054
已经抓起2871个连接
https://baike.baidu.com/item/%E5%BE%90%E8%89%AF/15946214#viewPageContent
已经抓起2872个连接
https://baike.baidu.com/item/%E7%8E%8B%E5%B0%94%E5%BE%B7/830891
已经抓起2873个连接
https://baike.baidu.com/item/%E6%AD%

https://baike.baidu.com/item/%E9%81%93%E6%A0%BC%E6%8B%89%E6%96%AF%C2%B7M%C2%B7%E6%A0%BC%E9%87%8C%E8%8A%AC/20202732
已经抓起2960个连接
https://baike.baidu.com/item/%E7%BA%A6%E7%BF%B0%E5%85%AD%E4%B8%96/9615980
已经抓起2961个连接
https://baike.baidu.com/item/%E7%AB%AF%E5%8D%88%E7%B2%BD/22291671
已经抓起2962个连接
https://baike.baidu.com/item/%E7%BA%BF%E6%80%A7%E4%BB%A3%E6%95%B0/800
已经抓起2963个连接
https://baike.baidu.com/item/%E5%BA%94%E7%A5%9E%E5%A4%A9%E7%9A%87/8951670
已经抓起2964个连接
https://baike.baidu.com/item/%E9%83%9D%E6%B3%BD%E5%98%89/4604649
已经抓起2965个连接
https://baike.baidu.com/item/%E8%BE%B9%E7%96%86/62163
已经抓起2966个连接
https://baike.baidu.com/item/%E6%A5%9A%E6%AD%A6%E7%8E%8B/7286034
已经抓起2967个连接
https://baike.baidu.com/item/%E6%96%87%E5%AD%A6/6437
已经抓起2968个连接
https://baike.baidu.com/item/%E5%BC%A0%E5%BB%B6/2964978#viewPageContent
已经抓起2969个连接
https://baike.baidu.com/item/%E8%83%8C%E9%B3%8D/7400804
已经抓起2970个连接
https://baike.baidu.com/item/%E8%80%81%E5%85%AC%E4%B8%87%E5%B2%81/1789701
已经抓起2971个连接
https://baike.baid

In [3]:
import pandas as pd
summary=pd.read_csv('unstruct_summary.csv',index_col=0)
# summary.drop_duplicates(subset=[0,1],keep)
summary.columns=['title','summary']
summary


# summary.drop_duplicates(['title'])

,title,summary
0,网络爬虫,\r\n网络爬虫（又被称为网页蜘蛛，网络机器人，在FOAF社区中间，更经常的称为网页追逐者）...
1,蠕虫,\r\n蠕虫是一种能够利用系统漏洞通过网络进行自我传播的恶意程序。它是利用网络进行复制和传播...
2,蠕虫,\r\n蠕虫，读作：rúchóng。意指许多相当小的、多少有点细长的像蚯蚓样的光裸而柔软的动...
3,www,\r\nWWW是环球信息网的缩写，（亦作“Web”、“WWW”、“'W3'”，英文全称为“W...
4,统一资源标识符,\r\n统一资源标识符（URI） 包括统一资源名称 (URN) 和统一资源定位器 (URL)...
5,英语,\r\n英语（English）是印欧语系-日耳曼语族下的语言，由26个字母组成，英文字母渊源...
6,www,\r\nWWW是环球信息网的缩写，（亦作“Web”、“WWW”、“'W3'”，英文全称为“W...
7,不列颠群岛,\r\n不列颠群岛是欧洲西北部的岛群。包括大不列颠和爱尔兰两个主要岛屿以及许多小岛在北海与大...
8,英语,\r\n《英语》是北京大学出版的图书，作者是段永华。\r\n
9,威尔士语,"\r\n威尔士语是凯尔特语的一种，在威尔士（Cymru）约有3,089,311人（2012年..."


In [7]:
import re
for i in range(len(summary)):
    a=re.compile(r'（.*?）')
    summary.iloc[i,1]=a.sub('',summary.iloc[i,1])
    summary.iloc[i,1]=summary.iloc[i,1].replace('\r\n','')

In [16]:
m=[]
k=0
for i in range(len(summary)):
    if summary.iloc[i,1]=="":
        k+=1
        m.append(i)
print(k,m)

91 [25, 51, 57, 59, 67, 68, 102, 139, 140, 164, 200, 203, 246, 287, 346, 451, 471, 476, 708, 765, 800, 880, 902, 910, 923, 1029, 1045, 1059, 1085, 1127, 1131, 1168, 1171, 1217, 1258, 1265, 1320, 1395, 1455, 1467, 1511, 1515, 1539, 1541, 1555, 1628, 1641, 1649, 1734, 1747, 1878, 1910, 1997, 2005, 2027, 2032, 2095, 2119, 2134, 2140, 2156, 2164, 2216, 2284, 2316, 2318, 2323, 2329, 2358, 2390, 2455, 2508, 2512, 2525, 2538, 2574, 2578, 2633, 2717, 2729, 2738, 2755, 2778, 2802, 2825, 2850, 2852, 2890, 2908, 2910, 2974]


In [23]:
# summary=summary.drop(m)
summary.to_csv('unstruct_summary.csv')

### 去除文字间的<>内容

In [9]:
import re
import sys

s='{通配符}你好，今天开学了{通配符},你好'
# print("s", s)
a1 = re.compile(r'\{.*?\}' )
d = a1.sub('',s)
print("d",d)
a1 = re.compile(r'\{[^}]*\}' )
d = a1.sub('',s)
print("d",d)

d 你好，今天开学了,你好
d 你好，今天开学了,你好


In [61]:
import re
from bs4 import BeautifulSoup

email_id_example = """<div class="para" label-module="para">网络爬虫（又被称为网页<a data-lemmaid="8135707" href="/item/%E8%9C%98%E8%9B%9B/8135707" target="_blank">蜘蛛</a>，网络机器人，在<a href="/item/FOAF" target="_blank">FOAF</a>社区中间，更经常的称为网页追逐者），是一种按照一定的规则，自动地抓取<a data-lemmaid="215515" href="/item/%E4%B8%87%E7%BB%B4%E7%BD%91/215515" target="_blank">万维网</a>信息的程序或者脚本。另外一些不常使用的名字还有<a data-lemmaid="9770178" href="/item/%E8%9A%82%E8%9A%81/9770178" target="_blank">蚂蚁</a>、自动索引、模拟程序或者<a data-lemmaid="4454380" href="/item/%E8%A0%95%E8%99%AB/4454380" target="_blank">蠕虫</a>。</div>
</div>
<div class="configModuleBanner">
</div><div class="basic-info cmn-clearfix">
<dl class="basicInfo-block basicInfo-left">
<dt class="basicInfo-item name">中文名</dt>
<dd class="basicInfo-item value">
网络爬虫
</dd>
<dt class="basicInfo-item name">外文名</dt>
<dd class="basicInfo-item value">
web crawler
</dd>
</dl><dl class="basicInfo-block basicInfo-right">
<dt class="basicInfo-item name">别    称</dt>
<dd class="basicInfo-item value">
网络蜘蛛
</dd>
"""

soup = BeautifulSoup(email_id_example,'lxml')
# emailid_regexp = re.compile(r"\w+@\w+\.\w+")　　　# regexp 表达式对象
first_email_id =re.findall('>.*?<',email_id_example)
first_email_id

['>网络爬虫（又被称为网页<',
 '>蜘蛛<',
 '>，网络机器人，在<',
 '>FOAF<',
 '>社区中间，更经常的称为网页追逐者），是一种按照一定的规则，自动地抓取<',
 '>万维网<',
 '>信息的程序或者脚本。另外一些不常使用的名字还有<',
 '>蚂蚁<',
 '>、自动索引、模拟程序或者<',
 '>蠕虫<',
 '>。<',
 '><',
 '>中文名<',
 '>外文名<',
 '><',
 '>别    称<']

In [1]:
import pandas as pd
c=[[1,2,3],[1,2,3]]
k=pd.DataFrame(c)
k

,0,1,2
0,1,2,3
1,1,2,3
